In [34]:
def experience_recup_multiSetPilot(MultiPilotPool,tache_time, nb, pause = 10, nb_exp=100,prob_init=None,step= None,type_recup = None, recup_tache_list= False, index_pilot= 0,nb_step=9):
    
    if prob_init is None:
        prob_init=[0.9,0.05,0.05]
    if step is None:
        step= [-0.1,0.075,0.025]
    if type_recup is None:
        type_recup = ["quad","other","exp"]
    
    time_total = sum(tache_time)
    l = len(MultiPilotPool)
    overload_list = np.zeros((l,nb_step,7))

    portion_pilot = np.zeros((l,nb_step,7))
    
    
    for i in range(nb_step):
        for j in range(nb_exp):
            prob = [a + i*b for a, b in zip(prob_init, step)]
            tachepool = generateTacheNb(nb, tache_time, prob)
            for indexPool, pilotPool1 in enumerate(MultiPilotPool):
                pp = [copy.deepcopy(pilotPool1) for i in range(7)]
                tp = [copy.deepcopy(tachepool) for i in range(7)]
                for recup in type_recup:
                    if recup == "quad":
                        for index in range(0,5):
                            for p in pp[index]:
                                p.r_b=index*0.5
                                p.r_a=(1-p.r_b)/15
                            ov=DistributionAvecRecuperationBas(pp[index],tp[index],methode = recup)
                            if(not sum(ov)):
                                portion_pilot[indexPool][i][index] += sum(tache_time[pp[index][index_pilot].tache])/time_total
                            if(sum(ov)):
                                overload_list[indexPool][i][index] += 1
                    if recup == "other":
                        index = 5
                        ov=DistributionAvecRecuperationBas(pp[index],tp[index],methode = recup)
                        if(not sum(ov)):
                            portion_pilot[indexPool][i][index] += sum(tache_time[pp[index][index_pilot].tache])/time_total
                        if(sum(ov)):
                            overload_list[indexPool][i][index] += 1
                    if recup == "exp":
                        index = 6
                        ov=DistributionAvecRecuperationBas(pp[index],tp[index],methode = recup)
                        if(not sum(ov)):
                            portion_pilot[indexPool][i][index] += sum(tache_time[pp[index][index_pilot].tache])/time_total
                        if(sum(ov)):
                            overload_list[indexPool][i][index] += 1
        for index in range(7):
            for indexPool in range(l):
                nb_ver = nb_exp
                if overload_list[indexPool][i][index] != nb_ver:
                    nb_no_overload = nb_ver - overload_list[indexPool][i][index]
                    portion_pilot[indexPool][i][index]  = portion_pilot[indexPool][i][index]/nb_no_overload
                else:
                    portion_pilot[indexPool][i][index]  = 1.01
                
    return overload_list,  portion_pilot

In [11]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from importlib import reload
import numpy as np
import copy
import matplotlib.pyplot as plt

In [6]:
import pilot_tache_distribution
reload(pilot_tache_distribution)
from pilot_tache_distribution import *
from util import *

In [35]:
seuil = [0,20, 70]
recuperation = 1

exps =[91,9]
exps_san_niveau1= [91]
workloads =[80,80]
workloads_san_niveau1 = [80]
tache_time = generateTachetime(100)
nb= 100
# 2 pilot H
pilotPool = generateFixPilot(exps,seuil,workloads,recuperation)
# 1 pilot H &1 pilotIA
pilotPool1= generateFixPilot(exps_san_niveau1,seuil,workloads_san_niveau1,recuperation)
AddIA(pilotPool1,0,seuil)
# 2 pilot H &1 pilotIA
pilotPool2 =pilotPool[:]
AddIA(pilotPool2,0,seuil)
multisetPilotPool = []
multisetPilotPool.append(pilotPool)
multisetPilotPool.append(pilotPool1)
multisetPilotPool.append(pilotPool2)
overload_list_all,  portion_pilot_all = experience_recup_multiSetPilot(multisetPilotPool, tache_time = tache_time,nb=nb,index_pilot = 1)

In [ ]:
def affichage_portion_pilot1(portion_pilot, n_bins= 100):
    
    stra_recup= ["1", "2","3", "4", "5","6","7"]
    #stra_recup= ["$1/15*x^2$", "$1/30*x^2+1/2*x$","$x$", "$-1/30*x^2+3/2*x$", "$-1/30*x^2+3/2*x$", "x-3", "exp",]
    colors = [(0, 1, 0), (1, 0, 0)]  # 绿色到红色
    nb_rang = 4
    submatrix = portion_pilot[:nb_rang,:]
    cmap = LinearSegmentedColormap.from_list('GreenToRed', colors, N=n_bins)

    # Affichage de la matrice
    fig, ax = plt.subplots(figsize=(14, 8))
    cax = ax.matshow(submatrix, cmap=cmap)

    # Suppression des axes
    #ax.axis('off')
    prob_list = []

    for  i in range(nb_rang):
        prob = [round(a + i*b,3) for a, b in zip(prob_init, step)]
        prob_list.append(prob)
    ax.set_xticks(np.arange(7))

    ax.set_xticklabels(stra_recup,rotation=0,  fontsize = 30,fontweight='bold')
    ax.tick_params(axis='both', which='major', labelsize=18)
    for (i, j), val in np.ndenumerate(portion_pilot):
        if  i < nb_rang:
            if val<=1:
                ax.text(j, i, f'{val*100:.000f}', ha='center', va='center', color='white', fontsize = 30, fontweight='bold')
            else:
                ax.text(j, i, '-', ha='center', va='center', color='white', fontsize = 30, fontweight='bold')

    # Adding a color bar
    ax.set_yticks(np.arange(nb_rang))
    ax.set_yticklabels(prob_list,fontsize = 30, fontweight='bold')

    ax.set_xlabel('type recup')  # Set the x-axis label
    ax.set_ylabel('Task Difficulty Distribution[Easy, Medium, and Hard]', fontsize = 15)  # Set the y-axis label
    colorbar =plt.colorbar(cax)
    colorbar.set_label('Number of tasks assigned to the operator ', fontsize = 20)
    plt.savefig("portion_pilot.pdf")
    plt.show()

In [ ]:
def affichage_recup1(overload_list, n_bins= 100,nb_step=9):
    #stra_recup= ["$1/15*x^2$", "$1/30*x^2+1/2*x$","$x$", "$-1/30*x^2+3/2*x$", "$-1/30*x^2+3/2*x$", "x-3", "exp",]
    stra_recup= ["1", "2","3", "4", "5","6","7"]
    colors = [(0, 1, 0), (1, 0, 0)]  # 绿色到红色
    nb_rang = 5
    cmap = LinearSegmentedColormap.from_list('GreenToRed', colors, N=n_bins)
    submatrix = overload_list[:nb_rang, :]
    # Affichage de la matrice
    fig, ax = plt.subplots(figsize=(14, 8))
    cax = ax.matshow(submatrix, cmap=cmap)

    # Suppression des axes
    #ax.axis('off')
    prob_list = []

    for  i in range(nb_rang):
        prob = [round(a + i*b,3) for a, b in zip(prob_init, step)]
        prob_list.append(prob)
    ax.set_xticks(np.arange(7))

    ax.set_xticklabels(stra_recup, rotation=0, fontsize = 18, fontweight='bold')
    for (i, j), val in np.ndenumerate(overload_list):
        if  i< nb_rang:
            ax.text(j, i, f'{val*1:.0f}', ha='center', va='center', color='white', fontsize = 30, fontweight='bold')

    # Adding a color bar
    ax.set_yticks(np.arange(nb_rang))
    ax.set_yticklabels(prob_list, fontsize = 20, fontweight='bold')

    ax.set_xlabel('type recup')  # Set the x-axis label
    ax.set_ylabel('Task Difficulty Distribution[Easy, Medium, and Hard]' , fontsize = 15)  # Set the y-axis label
    colorbar =plt.colorbar(cax)
    colorbar.set_label('Number of overloads in 100 experiments', fontsize = 20)
    plt.savefig("different_type_recup.pdf")
    plt.show()

In [ ]:

        prob_init=[0.9,0.05,0.05]

        step= [-0.1,0.075,0.025]

In [ ]:
for overload_list in overload_list_all:
    affichage_recup1(overload_list)

In [ ]:
for portion_pilot in portion_pilot_all:
    affichage_portion_pilot1(portion_pilot)
    